# Styling of biodiversity tiles

In this notebook we propose a method for rendering biodiversity layers to produce  the __Half-Earth Project__ map. The method combines a Jenks classification algorithm with a pre-defined palette to generate a spectrum of colours based on classified clusters.

Here we show a few examples using species richness and endemism data for birds and mammals. These datasets were developed by the Map of Life and can be accessed on their website https://mol.org/patterns/

## Load data

In [1]:
from urllib.request import urlopen
import zipfile
import os
import ee
import folium
import json
import pandas as pd
import geopandas as gpd
import IPython

ee.Initialize()

Global biodiversity patterns are provided in a 110 km equal-area gridded format. The global gridded shapefile is provided below:

In [3]:
# define data folder
data_path = 'data/'

# extract the zip file
z = zipfile.ZipFile(os.path.join(data_path,'360x114global_short.zip'), 'r')
z.extractall(data_path)

In [4]:
# Load map grids
grid = gpd.read_file("data/360x114global_short/360x114global_short.shp")
grid.head()

,HBWID,ID,X_COORD,Y_COORD,geometry
0,1,361,-179.5,79.11166,POLYGON ((-179.0000000004335 76.46375503182459...
1,2,362,-178.5,79.11166,POLYGON ((-178.0000000029935 76.46375503182459...
2,3,363,-177.5,79.11166,POLYGON ((-177.0000000055534 76.46375503182459...
3,4,364,-176.5,79.11166,POLYGON ((-176.0000000081132 76.46375503182459...
4,5,365,-175.5,79.11166,POLYGON ((-175.0000000106731 76.46375503182459...


## Download biodiversity data  from GEE

Biodiversity data is currently located in Earth Engine as we've been using this tool for prototyping. We will download 4 tables from EE corresponding to global richness and rarity values for birds and mammals. These tables will be used latter on for plotting

In [5]:
# Load and filter FeatureCollection tables
mol_facets = ee.FeatureCollection("users/guizarcou/mol_patterns_facet_map")

mammals_sr = mol_facets.filter(
    ee.Filter.eq('ptype','sr')).filter(
    ee.Filter.eq('taxa','mammals')).filter(
    ee.Filter.neq('pvalue', None))

mammals_se = mol_facets.filter(
    ee.Filter.eq('ptype','se')).filter(
    ee.Filter.eq('taxa','mammals')).filter(
    ee.Filter.neq('pvalue', None))

birds_sr = mol_facets.filter(
    ee.Filter.eq('ptype','sr')).filter(
    ee.Filter.eq('taxa','birds')).filter(
    ee.Filter.neq('pvalue', None))

birds_se = mol_facets.filter(
    ee.Filter.eq('ptype','se')).filter(
    ee.Filter.eq('taxa','birds')).filter(
    ee.Filter.neq('pvalue', None))

In [6]:
# define facets
facets = ["birds_se","birds_sr","mammals_se","mammals_sr"]


# download data tables
for f in facets:
    url = globals()[f].getDownloadURL(filetype='JSON',
                                selectors = ['hbwid','pvalue'],
                                filename =f)
    print("url: ", url)
    data = urlopen(url)

    geojson = os.path.join(data_path, "%s.geojson" % (f))

    with open(geojson, 'wb') as fp:
        while True:
            chunk = data.read()
            if not chunk: break
            fp.write(chunk)
    print('Download complete!')


url:  https://earthengine.googleapis.com/api/table?docid=4611f6104dc83fc63fcc1843f82dc9f8&token=aa1c8049b5bc37a7c9dbfb58db3f0336
Download complete!
url:  https://earthengine.googleapis.com/api/table?docid=20cd88278fc8181115c6df8df40c82a0&token=84b876ace1f295a835c16f9e1743eb8c
Download complete!
url:  https://earthengine.googleapis.com/api/table?docid=3537467988b541952faa9a7d4bbacccc&token=e35afebf6adb9a10a97068ec39305e99
Download complete!
url:  https://earthengine.googleapis.com/api/table?docid=4eddf3a322f52cb8fe8243886c5f7357&token=de8fd27443f7460419e0f0009da563f4
Download complete!


## Data preparation
Now we'll modify the structure of the downloaded tables so that the grid ids (__HBWID__ column) becomes the tables' unique idetifier.  

In [7]:
# define data tables dict
tables = dict()

for f in facets:
    # load GeoJSON
    gdf = gpd.read_file(os.path.join(data_path, "%s.geojson" % (f)))
    
    # Convert to DataFrame + filter columns of interest
    df = pd.DataFrame(gdf[['hbwid','pvalue']].values)

    # rename columns
    df.columns = ['HBWID','pvalue']
    
    # Convert IDs to integer
    df.HBWID = df['HBWID'].astype(int)
    
    # set HBWID as index
    df.set_index(keys='HBWID',inplace=True)
    
    # append to dictionary
    tables[f] = df    

# Preview data
tables['mammals_sr'].head()

,pvalue
HBWID,
28575,161.0
28576,155.0
28219,152.0
28218,168.0
28217,169.0


## Data clustering using the Jenks natural breaks method

We tested with a few approaches to add colours on the maps, including `equal distance`, `quantiles` and `jenks (natural)` breaks.

The Jenks classification method worked quite well when ploting richness and rarity layers as it allows to appreciate the differences in the distribution of the data (see examples at the end of the doc).

The sections below provide a workflow for generating map tiles using the Jenks classification method

In [8]:
from pysal.esda.mapclassify import Fisher_Jenks

In [9]:
# Generate Jenks classification
for f in facets:
    fj = Fisher_Jenks(tables[f]['pvalue'].values, k=100)
    tables[f]['class'] = fj.yb

In [10]:
# Preview
tables['mammals_sr'].head()

,pvalue,class
HBWID,,
28575,161.0,78
28576,155.0,76
28219,152.0,75
28218,168.0,81
28217,169.0,81


Define a dictionary object for each data table to retrieve biodiviersity valies per HWBID (these objects will be used in the `folium.GeoJson` mapping function)

In [11]:
# produce dict object with id:class
dic_pvals = dict()

for f in facets:
    df = tables[f]['class']
    df = df.to_dict()
    
     # append to dictionary
    dic_pvals[f] = df 

## Define colour scale

In [12]:
# Define colors
multiColPal = ['#FA6262','#EF9B61','#EEC86D','#AAF6A1','#39E8D2','#00B2E2','#7743E4']
multiColPal.reverse()

In [13]:
import branca.colormap as cm

# Generate master palette
pal = cm.LinearColormap(
    multiColPal)

pal

In [14]:
# palettes object
palettes = dict()

for f in facets:
    palettes[f] = pal.to_step(
        n=100,
        data= tables[f]['class'].values,
        method='linear',
        round_method='int')
    
# preview
palettes[facets[1]]

## Create GeoJSON objects for plotting
Finally we will generate GeoJSON objects that combine the gridded shapefile features with the classified values. Below I'm producing one file per facet/species since the reults are easier to load in the notebook for review, however a simlper approach would be to combine all the results in a single GeoJSON to serve in the app's backend.

In [16]:
geojsons = dict()

for f in facets:
    # Create HBWID column
    tables[f]['HBWID'] = tables[f].index

    # Merge GPD and DF objects
    gdf = grid[['HBWID','geometry']].merge(tables[f], on = 'HBWID')

    # Convert IDs to integer
    gdf['HBWID'] = gdf['HBWID'].astype(int)
    
    # Change ids to hbwid
    gdf.set_index(keys='HBWID',inplace=True)

    # Convert shapefiles to geojson
    geojsons[f] = json.loads(gdf.to_json())

# Plots

In [17]:
for f in facets:
    m = folium.Map(tiles='cartodbpositron',min_zoom=2,max_zoom=4)
    
    # generate styled geojson
    jsonOut = folium.GeoJson(
        geojsons[f],
        style_function=lambda feature: {
            'fillColor': palettes[f](dic_pvals[f][int(feature['id'])]),
            'fillOpacity':1,
            'color': '#ebebeb',
            'weight':0.05
        }
    ).add_to(m)
    
    # save
    html = "%s.html" % (f)
    m.save(html)

In [18]:
# load map
iframe = '<iframe src=' + "mammals_sr.html" + ' width=800 height=500></iframe>'
IPython.display.HTML(iframe)

## Mammals richness
![title](pics/mammals_sr.png)

In [86]:
# load map
iframe = '<iframe src=' + "mammals_se.html" + ' width=800 height=500></iframe>'
IPython.display.HTML(iframe)

## Mammals endemism
![title](pics/mammals_se.png)

In [87]:
# load map
iframe = '<iframe src=' + "birds_sr.html" + ' width=800 height=500></iframe>'
IPython.display.HTML(iframe)

## Birds richness
![title](pics/birds_sr.png)

In [88]:
# load map
iframe = '<iframe src=' + "birds_se.html" + ' width=800 height=500></iframe>'
IPython.display.HTML(iframe)

## Birds endemism
![title](pics/birds_se.png)